In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

In [2]:
%store -r X
%store -r y
%store -r X_test
%store -r y_test

In [3]:
# Định hình dữ liệu đầu vào
input_shape = (20, 128, 3)
input_layer = layers.Input(shape=input_shape)

# Thêm một chiều cho mỗi mẫu để biến đổi thành ảnh màu giả tạo
X_rgb = np.expand_dims(X, axis=-1)
X_rgb = np.repeat(X_rgb, 3, axis=-1) 

In [4]:
base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=input_layer)

# Tạo lớp kết nối đầy đủ
x = layers.Flatten()(base_model.output)

x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(5, activation='softmax')(x)  # Thay đổi số lớp đầu ra tùy theo bài toán của bạn
for layer in base_model.layers:
    layer.trainable = False
# Xây dựng mô hình hoàn chỉnh
model = Model(inputs=input_layer, outputs=x)

# In thông tin mô hình
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20, 128, 3)]         0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 26, 134, 3)           0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 10, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 10, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_rgb, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 165s 196ms/step - loss: 1.2744 - accuracy: 0.4825
Epoch 2/10
823/823 [==============================] - 158s 192ms/step - loss: 1.0876 - accuracy: 0.5691
Epoch 3/10
823/823 [==============================] - 152s 184ms/step - loss: 0.9996 - accuracy: 0.6098
Epoch 4/10
823/823 [==============================] - 137s 166ms/step - loss: 0.9633 - accuracy: 0.6242
Epoch 5/10
823/823 [==============================] - 140s 170ms/step - loss: 0.9152 - accuracy: 0.6455
Epoch 6/10
823/823 [==============================] - 144s 174ms/step - loss: 0.8913 - accuracy: 0.6587
Epoch 7/10
823/823 [==============================] - 153s 186ms/step - loss: 0.8708 - accuracy: 0.6609
Epoch 8/10
823/823 [==============================] - 141s 171ms/step - loss: 0.8565 - accuracy: 0.6728
Epoch 9/10
823/823 [==============================] - 139s 169ms/step - loss: 0.8549 - accuracy: 0.6760
Epoch 10/10
823/823 [==============================] - 139s 169m

In [6]:
# Thêm một chiều cho mỗi mẫu để biến đổi thành ảnh màu giả tạo
X_test_rgb = np.expand_dims(X_test, axis=-1)
X_test_rgb = np.repeat(X_test_rgb, 3, axis=-1) 

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test_rgb, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(X_test_rgb)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 77s 185ms/step - loss: 0.8340 - accuracy: 0.6809
Độ chính xác trên tập kiểm tra: 0.6809205412864685
412/412 [==============================] - 76s 182ms/step
Ma trận nhầm lẫn:
[[2676   14   65    1   86]
 [ 791  314  340    0  589]
 [ 144   18 2472    0  174]
 [ 165   48  155 1272  100]
 [ 426   86  998    1 2231]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       0.64      0.94      0.76      2842
           1       0.65      0.15      0.25      2034
           2       0.61      0.88      0.72      2808
           3       1.00      0.73      0.84      1740
           4       0.70      0.60      0.64      3742

    accuracy                           0.68     13166
   macro avg       0.72      0.66      0.64     13166
weighted avg       0.70      0.68      0.65     13166

